# How to use the Redis checkpointer for persistence

<div class="admonition tip">
    <p class="admonition-title">Prerequisites</p>
    <p>
        This guide assumes familiarity with the following:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    Persistence
                </a>
            </li>       
            <li>
                <a href="https://www.redis.io/">
                    Redis
                </a>
            </li>        
        </ul>
    </p>
</div> 

When creating LangGraph agents, you can also set them up so that they persist their state. This allows you to do things like interact with an agent multiple times and have it remember previous interactions.

This how-to guide shows how to use `Redis` as the backend for persisting checkpoint state using the [`langgraph-checkpoint-redis`](https://github.com/redis-developer/langgraph-redis) library.

For demonstration purposes we add persistence to the [pre-built create react agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent). 

In general, you can add a checkpointer to any custom graph that you build like this:

```python
from langgraph.graph import StateGraph

builder = StateGraph(....)
# ... define the graph
checkpointer = # postgres checkpointer (see examples below)
graph = builder.compile(checkpointer=checkpointer)
...
```

!!! info "Setup"
    You need to run `.setup()` once on your checkpointer to initialize the database before you can use it.

## Setup

First, let's install the required dependencies and ensure we have a Redis instance running.

Ensure you have a Redis server running. You can start one using Docker with:

```
docker run -d -p 6379:6379 redis:latest
```

Or install and run Redis locally according to your operating system's instructions.

In [1]:
# ruff: noqa: T201, I001, E501
from langgraph.checkpoint.redis import __lib_name__

print(f"langgraph-checkpoint-redis: {__lib_name__}")

langgraph-checkpoint-redis: langgraph-checkpoint-redis_v0.0.1


Next, let's install the required packages and set our API keys

In [2]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai
# %pip install -U langgraph-checkpoint-redis - already installed

In [3]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Define model and tools for the graph

In [4]:
from typing import Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.redis import RedisSaver


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

## Use sync connection

This sets up a synchronous connection to the database. 

Synchronous connections execute operations in a blocking manner, meaning each operation waits for completion before moving to the next one. The `REDIS_URI` is the Redis database connection URI:

In [5]:
REDIS_URI = "redis://redis:6379"

### With a connection string

This manages create a Redis client internal to the saver: 


In [6]:
with RedisSaver.from_conn_string(REDIS_URI) as checkpointer:
    # NOTE: you need to call .setup() the first time you're using your checkpointer
    checkpointer.setup()

    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "1"}}
    res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)
    checkpoint = checkpointer.get(config)

#### Examine the response

In [7]:
res

{'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='5acaae2b-cd12-48d2-84b3-fd8b6d20efb1'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iI1XPnrsC54rIPxBx01u0ozL', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7670bfa7-bee3-4a20-bbee-56a2a5139d2e-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_iI1XPnrsC54rIPxBx01u0ozL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, '

#### Examine the last checkpoint

In [8]:
checkpoint

{'type': 'json',
 'v': 1,
 'ts': '2025-02-04T14:50:40.204590+00:00',
 'id': '1efe3076-6e09-695a-8003-2c6b06cb6684',
 'channel_values': {'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='5acaae2b-cd12-48d2-84b3-fd8b6d20efb1'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iI1XPnrsC54rIPxBx01u0ozL', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7670bfa7-bee3-4a20-bbee-56a2a5139d2e-0', tool_calls=[{'name': 'g

### Bring your own client

You can pass a Redis client directly to the RedisSaver using the `redis_client` parameter:

In [9]:
from redis import Redis

client = Redis.from_url(REDIS_URI)

with RedisSaver.from_conn_string(redis_client=client) as checkpointer:
    # NOTE: you need to call .setup() the first time you're using your checkpointer
    # checkpointer.setup()
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "2"}}
    res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)

    checkpoint_tuple = checkpointer.get_tuple(config)

#### Retrieve a tuple containing a checkpoint and its associated data

In [10]:
checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efe3076-7d4f-6266-8003-6bc641be583a'}}, checkpoint={'type': 'json', 'v': 1, 'ts': '2025-02-04T14:50:41.805945+00:00', 'id': '1efe3076-7d4f-6266-8003-6bc641be583a', 'channel_values': {'messages': [HumanMessage(content="what's the weather in sf", additional_kwargs={}, response_metadata={}, id='e49447b0-4f49-46cd-8090-4ac01884fcaa'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mUggnSjGorTQ5Q97AzwwXzfm', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 57, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprin

#### Retrieve all the checkpoint tuples

In [11]:
checkpoint_tuples = list(checkpointer.list(config))
checkpoint_tuples

[CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efe3076-6e6e-61e9-bfff-0b96b56ad407'}}, checkpoint={'type': 'json', 'v': 1, 'ts': '2025-02-04T14:50:40.245802+00:00', 'id': '1efe3076-6e6e-61e9-bfff-0b96b56ad407', 'channel_values': {'__start__': {'messages': [['human', "what's the weather in sf"]]}}, 'channel_versions': {'__start__': '00000000000000000000000000000001.0.15048960283780277'}, 'versions_seen': {'__input__': {}}, 'pending_sends': []}, metadata={'source': 'input', 'writes': {'__start__': {'messages': [['human', "what's the weather in sf"]]}}, 'thread_id': '2', 'step': -1, 'parents': {}}, parent_config=None, pending_writes=[('eb162c2d-2490-eefa-c6af-198796893b66', 'messages', [['human', "what's the weather in sf"]]), ('eb162c2d-2490-eefa-c6af-198796893b66', 'start:agent', '__start__')]),
 CheckpointTuple(config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1efe3076-6e71-62cd-8000-5a5dd925cf31'}}, c

## Use async connection

This sets up an asynchronous connection to the database. 

Async connections allow non-blocking database operations. This means other parts of your application can continue running while waiting for database operations to complete. It's particularly useful in high-concurrency scenarios or when dealing with I/O-bound operations.

In [12]:
from langgraph.checkpoint.redis.aio import AsyncRedisSaver

async with AsyncRedisSaver.from_conn_string(REDIS_URI) as checkpointer:
    # NOTE: you need to call .setup() the first time you're using your checkpointer
    await checkpointer.asetup()

    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "4"}}
    res = await graph.ainvoke(
        {"messages": [("human", "what's the weather in nyc")]}, config
    )

    checkpoint = await checkpointer.aget(config)
    checkpoint_tuple = await checkpointer.aget_tuple(config)
    checkpoint_tuples = [c async for c in checkpointer.alist(config)]

14:50:41 redisvl.index.index INFO   Index already exists, not overwriting.
14:50:41 redisvl.index.index INFO   Index already exists, not overwriting.
14:50:41 redisvl.index.index INFO   Index already exists, not overwriting.


In [13]:
checkpoint

{'type': 'json',
 'v': 1,
 'ts': '2025-02-04T14:50:44.156243+00:00',
 'id': '1efe3076-93b9-6299-8003-1c23bc4a7784',
 'channel_values': {'messages': [HumanMessage(content="what's the weather in nyc", additional_kwargs={}, response_metadata={}, id='04549846-9a7d-40d9-aee3-e3fd34a7f5d8'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0JgOs4FlXVlPOPNn1gtzlS89', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-144a2e8d-e1b9-402f-bcaa-63bded726078-0', tool_calls=[{'name': 

In [14]:
checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1efe3076-93b9-6299-8003-1c23bc4a7784'}}, checkpoint={'type': 'json', 'v': 1, 'ts': '2025-02-04T14:50:44.156243+00:00', 'id': '1efe3076-93b9-6299-8003-1c23bc4a7784', 'channel_values': {'messages': [HumanMessage(content="what's the weather in nyc", additional_kwargs={}, response_metadata={}, id='04549846-9a7d-40d9-aee3-e3fd34a7f5d8'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0JgOs4FlXVlPOPNn1gtzlS89', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 58, 'total_tokens': 74, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerpr

In [15]:
checkpoint_tuples

[CheckpointTuple(config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1efe3076-7dc8-6f18-bfff-e117ab8033a3'}}, checkpoint={'type': 'json', 'v': 1, 'ts': '2025-02-04T14:50:41.855868+00:00', 'id': '1efe3076-7dc8-6f18-bfff-e117ab8033a3', 'channel_values': {'__start__': {'messages': [['human', "what's the weather in nyc"]]}}, 'channel_versions': {'__start__': '00000000000000000000000000000001.0.37688935776347043'}, 'versions_seen': {'__input__': {}}, 'pending_sends': []}, metadata={'source': 'input', 'writes': {'__start__': {'messages': [['human', "what's the weather in nyc"]]}}, 'thread_id': '4', 'step': -1, 'parents': {}}, parent_config=None, pending_writes=[('a88ef875-9fc3-ad3b-5b69-b5b7a8c18b78', 'messages', [['human', "what's the weather in nyc"]]), ('a88ef875-9fc3-ad3b-5b69-b5b7a8c18b78', 'start:agent', '__start__')]),
 CheckpointTuple(config={'configurable': {'thread_id': '4', 'checkpoint_ns': '', 'checkpoint_id': '1efe3076-7dcb-658b-8000-2c2b36012519'}}